### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [ ]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f'{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}'
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [ ]:
# Create an empty list to store the reviews
reviews_list = []


# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    page_query_url = query_url + f'&page={page}'
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_query_url).json()
        
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        reviews_list.extend([review for review in reviews['response']['docs']])

        # Print the page that was just retrieved
        print(f'Checked page {page}')

    except:
        # Print the page number that had no results then break from the loop
        print(f'Request failed at page {page}')
        break


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[0:5], indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df.head()

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df.head()


In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].to_list()
titles


### Access The Movie Database API

In [55]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [51]:
query_url = url + 'The Ottoman Lieutenant' + tmdb_key_string
movie = requests.get(query_url).json()
print(json.dumps(movie, indent=4))


{
    "page": 1,
    "results": [
        {
            "adult": false,
            "backdrop_path": "/5kmoreEiqaU0JoE8GZmLwLvAsJq.jpg",
            "genre_ids": [
                10749,
                18,
                10752
            ],
            "id": 433356,
            "original_language": "en",
            "original_title": "The Ottoman Lieutenant",
            "overview": "Lillie, a determined American woman, ventures overseas to join Dr. Jude at a remote medical mission in the Ottoman Empire (now Turkey). However, Lillie soon finds herself at odds with Jude and the mission\u2019s founder, Woodruff, when she falls for the titular military man, Ismail, just as the war is about to erupt.",
            "popularity": 13.671,
            "poster_path": "/c5E8KTDuGNdF4Lmv3OhsQ0W1zPw.jpg",
            "release_date": "2017-03-10",
            "title": "The Ottoman Lieutenant",
            "video": false,
            "vote_average": 6.173,
            "vote_count": 240
        }


In [68]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print(f'Sleeping at {request_counter} requests')
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    title_query_url = url + title + tmdb_key_string
    movie_minimal = requests.get(title_query_url).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = movie_minimal['results'][0]['id']

        # Make a request for a the full movie details
        id_query_url = f'https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}'

        # Execute "GET" request with url
        movie = requests.get(id_query_url).json()
        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in movie['spoken_languages']]
        
        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie['production_countries']]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({'title': title,
                                 'original_title': movie['original_title'],
                                 'budget': movie['budget'],
                                 'genre': genres,
                                 'language': movie['original_language'],
                                 'spoken_languages': spoken_languages,
                                 'homepage': movie['homepage'],
                                 'overview': movie['overview'],
                                 'popularity': movie['popularity'],
                                 'runtime': movie['runtime'],
                                 'revenue': movie['revenue'],
                                 'release_date': movie['release_date'],
                                 'vote_average': movie['vote_average'],
                                 'vote_count': movie['vote_count'],
                                 'production_countries': production_countries})
        print(f'Found {title}')

    except:    
        # Print out the title that was found
        print(f'{title} not found.')


Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found.
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
F

In [70]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[0:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genre": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 0.84,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 4.0,
        "vote_count": 2,
        "production_countries": [
            "Argentina"
        ]
    },
    {

In [ ]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
